# Obtención X

En este notebook se va a crear la matriz X, que es la que contiene las imágenes (se crea un dataset).

In [1]:
import cv2
import glob
import skimage


#Librerias contornos 
import numpy as np
from skimage.io import imread, imshow
from skimage.filters import prewitt_h,prewitt_v
from skimage import data, io
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.filters import threshold_otsu 
from skimage.feature import hog
from skimage import data, exposure

$devolverDiccionarioEtiquetas$

In [2]:
# $Sift$
#devuelve descriptor de Sift
def siftMaker(images00000_mod):
    sift = cv2.xfeatures2d.SIFT_create()
    kp, des = sift.detectAndCompute(images00000_mod,None)    
    #quizas falta un tercer parametro:  imagen original
    img=cv2.drawKeypoints(images00000_mod,kp, images00000_mod, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    #cv2.imwrite('sift_keypoints.jpg',img)
    return des


# $Surf$
def surfmaker(images00000_mod):
    surf = cv2.xfeatures2d.SURF_create(400)
    kp, des = surf.detectAndCompute(images00000_mod,None)
    return des

# $Fast$ + $Brief$
def fastbrief_Maker(images00000_mod):
    fast = cv2.FastFeatureDetector_create()

    # find and draw the keypoints
    kp2 = fast.detect(images00000_mod,None)
    img2 = cv2.drawKeypoints(images00000_mod, kp2, None, color=(255,0,0))

#     # Print all default params
#     print( "Threshold: {}".format(fast.getThreshold()) )
#     print( "nonmaxSuppression:{}".format(fast.getNonmaxSuppression()) )
#     print( "neighborhood: {}".format(fast.getType()) )
#     print( "Total Keypoints with nonmaxSuppression: {}".format(len(kp2)) )

    # cv2.imwrite('fast_true.png',img2)
#     cv2.imshow("test1", img2)

    # Disable nonmaxSuppression
    fast.setNonmaxSuppression(0)
    kp3 = fast.detect(img2,None)

#     print( "Total Keypoints without nonmaxSuppression: {}".format(len(kp3)) )

    img3 = cv2.drawKeypoints(img2, kp3, None, color=(255,0,0))
    #cv2.imshow("test", img3)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    # cv2.imwrite('fast_false.png',img3)
    brief = cv2.xfeatures2d.BriefDescriptorExtractor_create()

    # Compute descriptors
    kp, des = brief.compute(img3, kp3) #fast nonsup
    return des


# $ORB$
def orbMaker(images00000_mod):
    # Initiate ORB detector
    orb = cv2.ORB_create()

    # find the keypoints with ORB
    kp = orb.detect(images00000_mod,None)

    # compute the descriptors with ORB
    kp, des = orb.compute(images00000_mod, kp)

    # draw only keypoints location,not size and orientation
    img2 = cv2.drawKeypoints(images00000_mod, kp, None, color=(0,255,0), flags=0)
    #plt.imshow(img2), plt.show()
    return des

In [3]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores son el tipo de signals que contiene.
'''


def devolverDiccionarioEtiquetas():
    
    diccionarioValores = {}
    diccionarioValores['00000'] = 'VelocidadMaxima20'
    diccionarioValores['00001'] = 'VelocidadMaxima30'
    diccionarioValores['00002'] = 'VelocidadMaxima50'
    diccionarioValores['00003'] = 'VelocidadMaxima60'
    diccionarioValores['00004'] = 'VelocidadMaxima70'
    diccionarioValores['00005'] = 'VelocidadMaxima80'
    diccionarioValores['00006'] = 'FinVelocidadMaxima80'
    diccionarioValores['00007'] = 'VelocidadMaxima100'
    diccionarioValores['00008'] = 'VelocidadMaxima120'
    diccionarioValores['00009'] = 'AdelantamientoProhibido'
    diccionarioValores['00010'] = 'AdelantamientoProhibidoParaCamiones'

    diccionarioValores['00011'] = 'InterseccionConPrioridad'
    diccionarioValores['00012'] = 'CalzadaConPrioridad'
    diccionarioValores['00013'] = 'CedaElPaso'
    diccionarioValores['00014'] = 'DetencionObligatoria'
    diccionarioValores['00015'] = 'CirculacionProhibida'
    diccionarioValores['00016'] = 'EntradaProhibidaAVehiculosMercancias'
    diccionarioValores['00017'] = 'EntradaProhibida'
    diccionarioValores['00018'] = 'OtrosPeligros'
    diccionarioValores['00019'] = 'CurvaPeligrosaHaciaLaIzquierda'
    diccionarioValores['00020'] = 'CurvaPeligrosaHaciaLaDerecha'

    diccionarioValores['00021'] = 'CurvasPeligrosasHaciaLaIzquierda'
    diccionarioValores['00022'] = 'PerfilIrregular'
    diccionarioValores['00023'] = 'PavimentoDeslizante'
    diccionarioValores['00024'] = 'EstrechamientoCalzadaPorDerecha'
    diccionarioValores['00025'] = 'Obras'
    diccionarioValores['00026'] = 'Semaforos'
    diccionarioValores['00027'] = 'PasoDePeatones'
    diccionarioValores['00028'] = 'Kids'
    diccionarioValores['00029'] = 'EntradaProhibidasCiclos'
    diccionarioValores['00030'] = 'PavimentoDeslizanteNieveHielo'

    diccionarioValores['00031'] = 'PasoDeAnimalesEnLibertad'
    diccionarioValores['00032'] = 'FinDeProhibiciones'
    diccionarioValores['00033'] = 'SentidoObligatorioDerecha'
    diccionarioValores['00034'] = 'SentidoObligatorioIzquierda'
    diccionarioValores['00035'] = 'SentidoObligatorioArriba'
    diccionarioValores['00036'] = 'DirPermitidasArribaDerecha'
    diccionarioValores['00037'] = 'DirPermitidasArribaIzquierda'
    diccionarioValores['00038'] = 'PasoObligatorioDerecha'
    diccionarioValores['00039'] = 'PasoObligatorioizquierda'
    diccionarioValores['00040'] = 'InterseccionSentidoObligatorioGiratorio'


    diccionarioValores['00041'] = 'FinProhibicionAdelantamiento'
    diccionarioValores['00042'] = 'FinProhibicionAdelantamientoCamiones'
    
    return diccionarioValores

Probemos ahora el código....

In [4]:
d =  devolverDiccionarioEtiquetas()

for k,v in d.items():
    print(v)

VelocidadMaxima20
VelocidadMaxima30
VelocidadMaxima50
VelocidadMaxima60
VelocidadMaxima70
VelocidadMaxima80
FinVelocidadMaxima80
VelocidadMaxima100
VelocidadMaxima120
AdelantamientoProhibido
AdelantamientoProhibidoParaCamiones
InterseccionConPrioridad
CalzadaConPrioridad
CedaElPaso
DetencionObligatoria
CirculacionProhibida
EntradaProhibidaAVehiculosMercancias
EntradaProhibida
OtrosPeligros
CurvaPeligrosaHaciaLaIzquierda
CurvaPeligrosaHaciaLaDerecha
CurvasPeligrosasHaciaLaIzquierda
PerfilIrregular
PavimentoDeslizante
EstrechamientoCalzadaPorDerecha
Obras
Semaforos
PasoDePeatones
Kids
EntradaProhibidasCiclos
PavimentoDeslizanteNieveHielo
PasoDeAnimalesEnLibertad
FinDeProhibiciones
SentidoObligatorioDerecha
SentidoObligatorioIzquierda
SentidoObligatorioArriba
DirPermitidasArribaDerecha
DirPermitidasArribaIzquierda
PasoObligatorioDerecha
PasoObligatorioizquierda
InterseccionSentidoObligatorioGiratorio
FinProhibicionAdelantamiento
FinProhibicionAdelantamientoCamiones


In [5]:
def crearDiccionarioDatos():
    trafic_signals = {}
    #para cada tipo de señal creamos una lista con sus ejemplos, y se la asignamos a su etiqueta de nombre de carpeta
    for i in range(0,43):
        if i<10:
            folder = 'Dataset_traffic_sign/0000'+str(i)+'/*.png'
            tipo = '0000'+str(i)
        else:
            folder = 'Dataset_traffic_sign/000'+str(i)+'/*.png'
            tipo = '000'+str(i)
        
        images = [cv2.imread(file,0) for file in glob.glob(folder)]
        images_mod = [] 
        for im in images:
            #se aplica el resize
            images_mod.append(cv2.resize(im, (100, 100), interpolation = cv2.INTER_NEAREST))

        trafic_signals[tipo] = images_mod
    return trafic_signals

# Creamos nuestra matriz de datos
def creaMatrizCaracteristicas(trafic_signals, signals_types, carac_type):
    X = 0
    Y = 0
    
    for signal,images in trafic_signals.items():
        y = []
        x = []
        
        x = obtenerDesCaracteristicas(np.array(images), carac_type)
        
        for i in range(x.shape[0]):
            y.append(signals_types[signal])
            
        y = np.array(y).reshape(-1,1)
        
        if signal=='00000':
            Y = y
            X = x
        else:
            X = np.vstack((X,x))
            Y = np.vstack((Y,y))
    
    return X,Y

# Pasamos una np array de imagenes y vamos sacando sus descriptores 1 a 1
def obtenerDesCaracteristicas(images_o, carac_type):
    des = None
    
    for i in range(images_o.shape[0]):
        if carac_type=='sift':
            d = siftMaker(images_o[i])
        elif carac_type=='surf':
            d = surfmaker(images_o[i])
        elif carac_type=='fast_brief':
            d = fastbrief_Maker(images_o[i])
        elif carac_type=='orb':
            d = orbMaker(images_o[i])
        else:
            try:
                raise ValueError
            except ValueError:
                print("valor incorrecto")
                
        if d is not None:
            if des is None:
                des = d
            else:
                des = np.vstack((des,d))
    return des

In [10]:
trafic_signals = crearDiccionarioDatos()
signals_types = devolverDiccionarioEtiquetas()
carac_type = 'orb'

In [11]:
X, y = creaMatrizCaracteristicas(trafic_signals, signals_types, carac_type)

In [12]:
X.shape, y.shape

((22883, 32), (22883, 1))

In [13]:
X[0]


array([ 41, 162, 228,  55, 116,  30, 120,  48, 119, 237, 119, 205, 107,
        40, 150, 217,  30, 236, 229,  13, 172, 248,  77, 140,  78, 127,
       112, 129, 111, 142,  90,  93], dtype=uint8)